<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#!pip install polars

In [ ]:
import polars as pl
import random
import string

# Generacion de Datos

In [2]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""hngdl""",4,76.521722,"2013-04-18 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4105
2,"""qihza""",19,48.414261,"2014-12-01 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2010
3,"""xzjbn""",11,34.133727,"2017-09-12 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3066
4,"""eqwtf""",12,74.642879,"2016-03-16 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2309
5,"""nkxtc""",4,41.231704,"2014-06-06 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1790


In [37]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

### Esta es la respuesta correcta jeje
    

In [6]:


lazy_df = df.lazy()

resp = lazy_df.filter(pl.col('cantidad') < 5).with_columns((pl.col('cantidad') * pl.col('precio_unitario')).alias('total_venta')).drop('precio_unitario').group_by('producto').agg([ #agg se usa para agregar una columna con el total de la venta y la cantidad total vendida, aggs es una lista de operaciones que se aplican a las columnas
        pl.col('total_venta').sum().alias('total_vendido'), 
        pl.col('cantidad').sum().alias('cantidad_total_vendida')
    ]).sort('total_vendido', descending=True) #Se ordena el DataFrame por la columna total_vendido de forma descendente

resp.collect()

producto,total_vendido,cantidad_total_vendida
str,f64,i64
"""jqkcs""",397.144761,4
"""jknzs""",388.545899,4
"""zojxx""",367.071627,4
"""akcae""",359.953472,4
"""yanrn""",356.026327,4
…,…,…
"""yhjmn""",19.279604,1
"""janlh""",17.219368,1
"""itmfe""",13.88584,1


In [5]:
lazy_df = df.lazy()

resultado = ( #Gracias copilot por existir <3
    lazy_df.filter(pl.col('cantidad') < 5) #El filtro se hace sobre la columna cantidad para obtener los productos que se vendieron menos de 5 veces
    .with_columns((pl.col('cantidad') * pl.col('precio_unitario')).alias('total_venta')) #with_columns se usa para crear una nueva columna con el total de la venta 
    .drop('precio_unitario') #Se elimina la columna precio_unitario ya que no se necesita
    .group_by('producto').agg([ #agg se usa para agregar una columna con el total de la venta y la cantidad total vendida, aggs es una lista de operaciones que se aplican a las columnas
        pl.col('total_venta').sum().alias('total_vendido'), 
        pl.col('cantidad').sum().alias('cantidad_total_vendida')
    ]).sort('total_vendido', descending=True) #Se ordena el DataFrame por la columna total_vendido de forma descendente
)



# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)

shape: (201, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ jqkcs    ┆ 397.144761    ┆ 4                      │
│ jknzs    ┆ 388.545899    ┆ 4                      │
│ zojxx    ┆ 367.071627    ┆ 4                      │
│ akcae    ┆ 359.953472    ┆ 4                      │
│ yanrn    ┆ 356.026327    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ yhjmn    ┆ 19.279604     ┆ 1                      │
│ janlh    ┆ 17.219368     ┆ 1                      │
│ itmfe    ┆ 13.88584      ┆ 1                      │
│ altac    ┆ 13.561419     ┆ 1                      │
│ mkpwf    ┆ 10.201399     ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [7]:
# Tu codigo aqui
resultado_df.write_parquet("resultados.parquet")



# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [8]:
# Tu codigo aqui
df_resultados = pl.read_parquet("resultados.parquet")
print(df_resultados)

shape: (201, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ jqkcs    ┆ 397.144761    ┆ 4                      │
│ jknzs    ┆ 388.545899    ┆ 4                      │
│ zojxx    ┆ 367.071627    ┆ 4                      │
│ akcae    ┆ 359.953472    ┆ 4                      │
│ yanrn    ┆ 356.026327    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ yhjmn    ┆ 19.279604     ┆ 1                      │
│ janlh    ┆ 17.219368     ┆ 1                      │
│ itmfe    ┆ 13.88584      ┆ 1                      │
│ altac    ┆ 13.561419     ┆ 1                      │
│ mkpwf    ┆ 10.201399     ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘
